In [10]:
#import packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from keras.preprocessing import image
from keras.models import model_from_json
import flask
import PIL
from PIL import Image
from flask import Flask, request, json, jsonify
import io
import os

In [2]:
tf.__version__

'2.7.0'

In [3]:
#Loading the model
def modelAct():
    json_file = open('mnist.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights("mnist.h5")
    print('model loaded successfully')
    return model

In [4]:
#Transform Loading
def transform_image(pillow_image):
    data = np.asarray(pillow_image)
    data = data / 255.0
    data = data[np.newaxis, ..., np.newaxis]
    # --> [1, x, y, 1]
    data = tf.image.resize(data, [28, 28])
    return data


In [5]:
#Predict
def predict (img_data):
    model = modelAct()
    prediction = None
    result = None
    result = model.predict(img_data)
    print(np.max(result))
    print(result[0])
    for i in range(10):
        if result[0][i] == np.max(result):
            prediction = i
            
    return prediction

In [6]:
# pillow_img = Image.open("C:/Users/Dinesh/Desktop/img_11.jpg").convert('L')
# tensor = transform_image(pillow_img)
# resultTest = predict(tensor)
# print(resultTest)

In [7]:
app = Flask(__name__)

@app.route ("/", methods = ["GET","POST"])
def index():
    welcomeMsg = "Welcome to MNIST digit analysis, by DINESHCHANDAR RAVICHANDRAN \n contact me @: dravich@g.g.clemson.edu\n"
    if request.method == "POST":
        file = request.files['file']
        if file is None or file.filename == "":
            return jsonify ({"error": "No input found!"})
        else:
           
            try:
                image_bytes = file.read()
                pillow_img = Image.open(io.BytesIO(image_bytes)).convert('L')
                tensor = transform_image(pillow_img)
                prediction = predict(tensor)
                result = {"Prediction": int(prediction)}
                print(result)
                return jsonify (welcomeMsg,result)
            except Exception as e:
                return jsonify (welcomeMsg,{"error":str(e)})
    else:
        return "Health check: OK"


In [8]:
# if __name__ == "__main__":
#     app.run (debug = False)

In [12]:
if __name__ == "__main__":
 port = int(os.environ.get('PORT', 5000))
 app.run(debug=False, host='0.0.0.0', port=port)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://198.21.214.215:5000/ (Press CTRL+C to quit)


model loaded successfully


198.21.214.215 - - [01/Mar/2022 17:54:29] "POST /?files HTTP/1.1" 200 -


0.9999924
[1.3896469e-12 1.2993699e-14 3.5029000e-12 1.1937790e-06 1.2706877e-13
 9.9999237e-01 2.0406250e-09 3.9402344e-12 2.2929518e-07 6.2280678e-06]
{'Prediction': 5}
